# AFAS SB Baseline Carbon Emissions Assessment (RQ1)

This notebook presents the baseline calculation of carbon emissions for AFAS SB using the Software Carbon Intensity (SCI) specification. The analysis establishes the production system's baseline carbon footprint through Azure monitoring data collected over a one-week period (February 3-9, 2025).

**Research Question 1**: What is the baseline rate of carbon emissions of AFAS SB in production?

The analysis encompasses 46 instances across 12 resource groups, including VM Scale Sets, Virtual Machines, and SQL Elastic Pools deployed on Microsoft Azure Service Fabric.

## 1. Setup and Data Loading

Loading the processed SCI calculation results and instance configuration data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import os

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
plt.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 11,
    'figure.figsize': (12, 8),
    'figure.dpi': 100,
    'savefig.dpi': 150,
    'savefig.bbox': 'tight'
})

if not os.path.exists('figures'):
    os.makedirs('figures')

if not os.path.exists('baseline_results'):
    os.makedirs('baseline_results')

print("Environment initialized successfully")

In [ ]:
summary_df = pd.read_csv('sci_outputs_summary.csv')
instance_assignments = pd.read_csv('azure_data/static_data/instance_assignments.csv')
instance_specs = pd.read_csv('azure_data/static_data/instance_types_specs.csv')

print(f"Loaded SCI summary data: {len(summary_df)} instances")
print(f"Instance assignments: {len(instance_assignments)} records")
print(f"Instance specifications: {len(instance_specs)} types")

print("\nSCI Summary Data Structure:")
print(summary_df.head())
print("\nColumns:", summary_df.columns.tolist())

## 2. Data Preprocessing and Categorization

Extracting service types and resource groups from instance names based on the AFAS SB infrastructure taxonomy.

In [ ]:
def extract_resource_group(instance_name):
    if 'Clstr003' in instance_name:
        return 'Cluster003-RG'
    elif 'Clstr004' in instance_name:
        return 'Cluster004-RG'
    elif 'Clstr008' in instance_name:
        return 'Cluster008-RG'
    elif 'Clstr009' in instance_name:
        return 'Cluster009-RG'
    elif 'Dbg002-SS' in instance_name:
        return 'DebugCluster002-RG'
    elif 'DEBUG001' in instance_name:
        return 'SqlServerDebug001-RG'
    elif 'Logging-VM' in instance_name:
        return 'LoggingService-RG'
    elif 'Elastic-VM' in instance_name:
        return 'LoggingService-RG'
    elif 'proxy001L-SS' in instance_name:
        return 'Proxy001-RG'
    elif 'SvcProxy-VM' in instance_name:
        return 'ServicesProxy-RG'
    elif 'SERVER002' in instance_name:
        return 'SqlServer002-RG'
    elif 'SERVER003' in instance_name:
        return 'SqlServer003-RG'
    elif 'AFASSERVICESPOOL001' in instance_name or 'NEXT' in instance_name:
        return 'SqlServer001-RG'
    else:
        return 'Other'

def extract_service_type(instance_name):
    if '-SS_' in instance_name:
        return 'VM Scale Set'
    elif '-VM' in instance_name:
        return 'Virtual Machine'
    elif 'POOL' in instance_name or 'NEXTGP' in instance_name or 'DEBUG001' in instance_name:
        return 'SQL Elastic Pool'
    else:
        return 'Other'

summary_df['resource_group'] = summary_df['_instance'].apply(extract_resource_group)
summary_df['service_type'] = summary_df['_instance'].apply(extract_service_type)

print("Resource Group Distribution:")
resource_group_counts = summary_df['resource_group'].value_counts()
print(resource_group_counts)

print("\nService Type Distribution:")
service_type_counts = summary_df['service_type'].value_counts()
print(service_type_counts)

print(f"\nTotal instances processed: {len(summary_df)}")
print(f"Total resource groups: {summary_df['resource_group'].nunique()}")
print(f"Total service types: {summary_df['service_type'].nunique()}")

## 3. System-Wide Baseline Metrics

Calculating aggregate carbon emissions metrics for the entire AFAS SB production system.

In [ ]:
total_energy = summary_df['energy'].sum()
total_carbon = summary_df['carbon'].sum()
total_instances = len(summary_df)
time_period_hours = 168
system_sci_per_instance_hour = total_carbon / (total_instances * time_period_hours)

mean_instance_sci = summary_df['sci'].mean()
median_instance_sci = summary_df['sci'].median()
std_instance_sci = summary_df['sci'].std()
min_instance_sci = summary_df['sci'].min()
max_instance_sci = summary_df['sci'].max()

baseline_metrics = {
    'Total Energy Consumption (kWh)': total_energy,
    'Total Carbon Emissions (gCO2e)': total_carbon,
    'Total Instances': total_instances,
    'System SCI (gCO2e per instance-hour)': system_sci_per_instance_hour,
    'Mean Instance SCI (gCO2e/h)': mean_instance_sci,
    'Median Instance SCI (gCO2e/h)': median_instance_sci,
    'Std Instance SCI (gCO2e/h)': std_instance_sci,
    'Min Instance SCI (gCO2e/h)': min_instance_sci,
    'Max Instance SCI (gCO2e/h)': max_instance_sci
}

print("AFAS SB System-Wide Baseline Carbon Metrics")
print("=" * 50)
for metric, value in baseline_metrics.items():
    if 'Total' in metric and 'Instances' not in metric:
        print(f"{metric}: {value:.2f}")
    elif 'Instances' in metric:
        print(f"{metric}: {int(value)}")
    else:
        print(f"{metric}: {value:.3f}")

baseline_system_df = pd.DataFrame([
    {'Metric': k, 'Value': v} for k, v in baseline_metrics.items()
])
baseline_system_df.to_csv('baseline_results/baseline_system_metrics.csv', index=False)
print("\nSystem metrics exported to baseline_results/baseline_system_metrics.csv")

## 4. Service Type Analysis

Analyzing carbon intensity patterns across different service types in the AFAS SB infrastructure.

In [ ]:
service_type_stats = summary_df.groupby('service_type').agg({
    'sci': ['count', 'mean', 'median', 'std', 'min', 'max'],
    'energy': ['sum', 'mean'],
    'carbon': ['sum', 'mean']
})

service_type_stats.columns = ['_'.join(col).strip() for col in service_type_stats.columns.values]
service_type_stats = service_type_stats.reset_index()

service_type_stats['energy_percentage'] = (service_type_stats['energy_sum'] / total_energy) * 100
service_type_stats['carbon_percentage'] = (service_type_stats['carbon_sum'] / total_carbon) * 100

service_type_stats = service_type_stats.sort_values('sci_mean', ascending=False)

print("Service Type Analysis - SCI Statistics")
print("=" * 50)
for _, row in service_type_stats.iterrows():
    print(f"{row['service_type']}:")
    print(f"  Instances: {int(row['sci_count'])}")
    print(f"  Mean SCI: {row['sci_mean']:.3f} gCO2e/h")
    print(f"  Median SCI: {row['sci_median']:.3f} gCO2e/h")
    print(f"  Std SCI: {row['sci_std']:.3f} gCO2e/h")
    print(f"  Range: {row['sci_min']:.3f} - {row['sci_max']:.3f} gCO2e/h")
    print(f"  Carbon contribution: {row['carbon_percentage']:.1f}%")
    print()

service_type_stats.to_csv('baseline_results/service_type_analysis.csv', index=False)
print("Service type analysis exported to baseline_results/service_type_analysis.csv")

## 5. Resource Group Analysis

Examining carbon intensity distribution across resource groups to identify high-impact components.

In [ ]:
resource_group_stats = summary_df.groupby('resource_group').agg({
    'sci': ['count', 'mean', 'median', 'std', 'min', 'max'],
    'energy': ['sum', 'mean'],
    'carbon': ['sum', 'mean']
})

resource_group_stats.columns = ['_'.join(col).strip() for col in resource_group_stats.columns.values]
resource_group_stats = resource_group_stats.reset_index()

resource_group_stats['energy_percentage'] = (resource_group_stats['energy_sum'] / total_energy) * 100
resource_group_stats['carbon_percentage'] = (resource_group_stats['carbon_sum'] / total_carbon) * 100

resource_group_stats = resource_group_stats.sort_values('carbon_percentage', ascending=False)

print("Resource Group Analysis - Carbon Contribution Ranking")
print("=" * 60)
for i, (_, row) in enumerate(resource_group_stats.iterrows(), 1):
    print(f"{i}. {row['resource_group']}:")
    print(f"   Instances: {int(row['sci_count'])}")
    print(f"   Mean SCI: {row['sci_mean']:.3f} gCO2e/h")
    print(f"   Carbon contribution: {row['carbon_percentage']:.1f}%")
    print(f"   SCI range: {row['sci_min']:.3f} - {row['sci_max']:.3f} gCO2e/h")
    print()

top_3_contributors = resource_group_stats.head(3)
top_3_carbon_percentage = top_3_contributors['carbon_percentage'].sum()
print(f"Top 3 resource groups contribute {top_3_carbon_percentage:.1f}% of total carbon emissions")

resource_group_stats.to_csv('baseline_results/resource_group_analysis.csv', index=False)
print("\nResource group analysis exported to baseline_results/resource_group_analysis.csv")

## 6. SCI Distribution Analysis

Analyzing the statistical distribution of SCI scores across all instances.

In [ ]:
sci_scores = summary_df['sci']
quartiles = sci_scores.quantile([0.25, 0.5, 0.75])
iqr = quartiles[0.75] - quartiles[0.25]
cv = (sci_scores.std() / sci_scores.mean()) * 100

distribution_stats = {
    'Count': len(sci_scores),
    'Mean': sci_scores.mean(),
    'Median': sci_scores.median(),
    'Standard Deviation': sci_scores.std(),
    'Coefficient of Variation (%)': cv,
    'Minimum': sci_scores.min(),
    'Q1 (25th percentile)': quartiles[0.25],
    'Q3 (75th percentile)': quartiles[0.75],
    'Maximum': sci_scores.max(),
    'Interquartile Range': iqr,
    'Range': sci_scores.max() - sci_scores.min()
}

print("SCI Score Distribution Analysis")
print("=" * 40)
for stat, value in distribution_stats.items():
    if 'Count' in stat:
        print(f"{stat}: {int(value)}")
    else:
        print(f"{stat}: {value:.3f}")

percentile_breakdown = []
for percentile in [10, 25, 50, 75, 90, 95, 99]:
    value = sci_scores.quantile(percentile/100)
    percentile_breakdown.append({
        'Percentile': f"P{percentile}",
        'SCI_Score': value
    })

percentile_df = pd.DataFrame(percentile_breakdown)
print("\nPercentile Breakdown:")
for _, row in percentile_df.iterrows():
    print(f"{row['Percentile']}: {row['SCI_Score']:.3f} gCO2e/h")

distribution_df = pd.DataFrame([
    {'Statistic': k, 'Value': v} for k, v in distribution_stats.items()
])
distribution_df.to_csv('baseline_results/sci_distribution_analysis.csv', index=False)
percentile_df.to_csv('baseline_results/sci_percentile_breakdown.csv', index=False)
print("\nDistribution analysis exported to baseline_results/sci_distribution_analysis.csv")
print("Percentile breakdown exported to baseline_results/sci_percentile_breakdown.csv")

## 7. Performance Benchmarking

Identifying highest and lowest performing instances for optimization targeting.

In [ ]:
top_5_highest = summary_df.nlargest(5, 'sci')[['_instance', 'sci', 'resource_group', 'service_type']]
top_5_lowest = summary_df.nsmallest(5, 'sci')[['_instance', 'sci', 'resource_group', 'service_type']]

print("Highest Carbon Intensity Instances (Top 5)")
print("=" * 45)
for i, (_, row) in enumerate(top_5_highest.iterrows(), 1):
    print(f"{i}. {row['_instance']}")
    print(f"   SCI: {row['sci']:.3f} gCO2e/h")
    print(f"   Resource Group: {row['resource_group']}")
    print(f"   Service Type: {row['service_type']}")
    print()

print("Lowest Carbon Intensity Instances (Top 5)")
print("=" * 45)
for i, (_, row) in enumerate(top_5_lowest.iterrows(), 1):
    print(f"{i}. {row['_instance']}")
    print(f"   SCI: {row['sci']:.3f} gCO2e/h")
    print(f"   Resource Group: {row['resource_group']}")
    print(f"   Service Type: {row['service_type']}")
    print()

performance_ratio = top_5_highest['sci'].mean() / top_5_lowest['sci'].mean()
print(f"Performance Gap: {performance_ratio:.1f}x difference between highest and lowest performers")

top_5_highest.to_csv('baseline_results/worst_performers.csv', index=False)
top_5_lowest.to_csv('baseline_results/best_performers.csv', index=False)
print("\nPerformance analysis exported to baseline_results/worst_performers.csv and best_performers.csv")

## 8. Carbon Intensity Treemap Visualization

Creating a comprehensive treemap showing the hierarchical relationship between service types, resource groups, and their carbon intensity contributions.

In [ ]:
service_type_colors = {
    'VM Scale Set': '#2E86AB',
    'SQL Elastic Pool': '#A23B72', 
    'Virtual Machine': '#F18F01'
}

treemap_data = []
for service_type in summary_df['service_type'].unique():
    service_subset = summary_df[summary_df['service_type'] == service_type]
    
    for resource_group in service_subset['resource_group'].unique():
        rg_subset = service_subset[service_subset['resource_group'] == resource_group]
        total_sci = rg_subset['sci'].sum()
        
        treemap_data.append({
            'service_type': service_type,
            'resource_group': resource_group,
            'sci_total': total_sci,
            'instance_count': len(rg_subset),
            'color': service_type_colors[service_type]
        })

treemap_df = pd.DataFrame(treemap_data)
treemap_df = treemap_df.sort_values(['service_type', 'sci_total'], ascending=[False, True])

sizes = treemap_df['sci_total'].tolist()
colors = treemap_df['color'].tolist()

def create_smart_label(row, max_size):
    size_ratio = row['sci_total'] / max_size
    rg_name = row['resource_group']
    sci_value = row['sci_total']
    
    name_mapping = {
        'Cluster003-RG': 'Cluster003',
        'Cluster004-RG': 'Cluster004', 
        'Cluster008-RG': 'Cluster008',
        'Cluster009-RG': 'Cluster009',
        'DebugCluster002-RG': 'DebugCluster002',
        'SqlServerDebug001-RG': 'SqlDebug001',
        'LoggingService-RG': 'LoggingService',
        'Proxy001-RG': 'Proxy001',
        'ServicesProxy-RG': 'ServicesProxy',
        'SqlServer001-RG': 'SqlServer001',
        'SqlServer002-RG': 'SqlServer002',
        'SqlServer003-RG': 'SqlServer003'
    }
    
    display_name = name_mapping.get(rg_name, rg_name)
    
    return f"{display_name}\n{sci_value:.1f} gCO2e/h"


max_size = max(sizes)
labels = [create_smart_label(row, max_size) for _, row in treemap_df.iterrows()]

fig = plt.figure(figsize=(16, 10))
ax = fig.add_subplot(111)

squarify.plot(
    sizes=sizes,
    label=labels,
    color=colors,
    alpha=0.8,
    ax=ax,
    text_kwargs={
        'fontsize': 9,
        'weight': 'bold',
        'color': 'white'
    },
    bar_kwargs={
        'linewidth': 2,
        'edgecolor': 'white'
    }
)

legend_elements = [
    plt.Rectangle((0,0),1,1, facecolor=color, alpha=0.8, label=service_type)
    for service_type, color in service_type_colors.items()
]
ax.legend(
    handles=legend_elements,
    loc='lower center',
    bbox_to_anchor=(0.5, -0.05),
    ncol=len(legend_elements),
    fontsize=12,
    frameon=False
)


ax.axis('off')

plt.subplots_adjust(right=0.85)
plt.savefig('figures/carbon_intensity_treemap.png', dpi=150, bbox_inches='tight', 
            facecolor='white', edgecolor='none', pad_inches=0.2)
plt.show()

treemap_df.to_csv('baseline_results/treemap_data.csv', index=False)
print("Treemap visualization saved to figures/carbon_intensity_treemap.png")
print("Treemap data exported to baseline_results/treemap_data.csv")

## 9. Comprehensive Summary

Consolidating all baseline analysis results into a comprehensive summary for RQ1.

In [ ]:
rq1_summary = {
    'baseline_assessment': {
        'total_energy_kwh': total_energy,
        'total_carbon_gco2e': total_carbon,
        'system_sci_per_instance_hour': system_sci_per_instance_hour,
        'instances_analyzed': total_instances,
        'resource_groups': summary_df['resource_group'].nunique(),
        'service_types': summary_df['service_type'].nunique()
    },
    'sci_distribution': {
        'mean_sci': mean_instance_sci,
        'median_sci': median_instance_sci,
        'std_sci': std_instance_sci,
        'min_sci': min_instance_sci,
        'max_sci': max_instance_sci,
        'coefficient_variation_percent': cv
    },
    'service_type_insights': {
        'highest_intensity_service': service_type_stats.iloc[0]['service_type'],
        'highest_intensity_sci': service_type_stats.iloc[0]['sci_mean'],
        'largest_contributor_service': service_type_stats.sort_values('carbon_percentage', ascending=False).iloc[0]['service_type'],
        'largest_contribution_percent': service_type_stats.sort_values('carbon_percentage', ascending=False).iloc[0]['carbon_percentage']
    },
    'resource_group_insights': {
        'highest_contributing_rg': resource_group_stats.iloc[0]['resource_group'],
        'highest_contribution_percent': resource_group_stats.iloc[0]['carbon_percentage'],
        'top_3_contribution_percent': top_3_carbon_percentage,
        'performance_gap_ratio': performance_ratio
    }
}

print("RQ1 COMPREHENSIVE SUMMARY")
print("=" * 50)
print("\nBaseline Assessment:")
print(f"  System-wide SCI: {rq1_summary['baseline_assessment']['system_sci_per_instance_hour']:.3f} gCO2e per instance-hour")
print(f"  Total carbon emissions: {rq1_summary['baseline_assessment']['total_carbon_gco2e']:.0f} gCO2e")
print(f"  Total energy consumption: {rq1_summary['baseline_assessment']['total_energy_kwh']:.2f} kWh")
print(f"  Infrastructure scope: {rq1_summary['baseline_assessment']['instances_analyzed']} instances across {rq1_summary['baseline_assessment']['resource_groups']} resource groups")

print("\nSCI Score Distribution:")
print(f"  Mean: {rq1_summary['sci_distribution']['mean_sci']:.3f} gCO2e/h")
print(f"  Median: {rq1_summary['sci_distribution']['median_sci']:.3f} gCO2e/h")
print(f"  Standard deviation: {rq1_summary['sci_distribution']['std_sci']:.3f} gCO2e/h")
print(f"  Range: {rq1_summary['sci_distribution']['min_sci']:.3f} - {rq1_summary['sci_distribution']['max_sci']:.3f} gCO2e/h")
print(f"  Coefficient of variation: {rq1_summary['sci_distribution']['coefficient_variation_percent']:.1f}%")

print("\nKey Insights:")
print(f"  Highest intensity service type: {rq1_summary['service_type_insights']['highest_intensity_service']} ({rq1_summary['service_type_insights']['highest_intensity_sci']:.3f} gCO2e/h)")
print(f"  Largest carbon contributor: {rq1_summary['service_type_insights']['largest_contributor_service']} ({rq1_summary['service_type_insights']['largest_contribution_percent']:.1f}% of total)")
print(f"  Top resource group: {rq1_summary['resource_group_insights']['highest_contributing_rg']} ({rq1_summary['resource_group_insights']['highest_contribution_percent']:.1f}% of total)")
print(f"  Top 3 resource groups contribute: {rq1_summary['resource_group_insights']['top_3_contribution_percent']:.1f}% of total emissions")
print(f"  Performance gap: {rq1_summary['resource_group_insights']['performance_gap_ratio']:.1f}x between best and worst performers")

summary_records = []
for category, metrics in rq1_summary.items():
    for metric, value in metrics.items():
        summary_records.append({
            'Category': category,
            'Metric': metric,
            'Value': value
        })

comprehensive_summary_df = pd.DataFrame(summary_records)
comprehensive_summary_df.to_csv('baseline_results/rq1_comprehensive_summary.csv', index=False)
print("\nComprehensive summary exported to baseline_results/rq1_comprehensive_summary.csv")

## 10. Results Export Summary

Overview of all generated analysis outputs and their locations.

In [ ]:
print("BASELINE CALCULATION ANALYSIS COMPLETE")
print("=" * 50)
print("\nGenerated CSV Files:")
csv_files = [
    'baseline_system_metrics.csv',
    'service_type_analysis.csv', 
    'resource_group_analysis.csv',
    'sci_distribution_analysis.csv',
    'sci_percentile_breakdown.csv',
    'worst_performers.csv',
    'best_performers.csv',
    'treemap_data.csv',
    'rq1_comprehensive_summary.csv'
]

for csv_file in csv_files:
    print(f"  - baseline_results/{csv_file}")

print("\nGenerated Visualizations:")
print("  - figures/carbon_intensity_treemap.png")

print("\nBaseline SCI Score for AFAS SB Production System:")
print(f"  {system_sci_per_instance_hour:.3f} gCO2e per instance-hour")
print(f"\nThis baseline establishes the reference point for evaluating")
print(f"configuration optimization strategies in subsequent research questions.")

print(f"\nAnalysis completed successfully. All results exported to baseline_results/ directory.")